In [1]:
import AdventOfCode

In [2]:
input <- day 16

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as C
import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as BC
import qualified Data.Vector as V

parseRegisters :: C.Parser (V.Vector Int)
parseRegisters = do
    C.string "["
    a <- C.decimal
    C.string ", "
    b <- C.decimal
    C.string ", "
    c <- C.decimal
    C.string ", "
    d <- C.decimal
    C.string "]"
    pure $ V.fromList [a, b, c, d]

(part1, part2) = BC.breakSubstring "\n\n\n" input

In [4]:
parseInstr :: C.Parser (V.Vector Int)
parseInstr = do
    a <- C.decimal
    C.skipSpace
    b <- C.decimal
    C.skipSpace
    c <- C.decimal
    C.skipSpace
    d <- C.decimal
    C.skipSpace
    pure $ V.fromList [a,b,c,d]

parseSample = do
    before <- "Before:" *> C.skipSpace *> parseRegisters
    C.skipSpace
    instr <- parseInstr
    C.skipSpace
    after <- "After:" *> C.skipSpace *> parseRegisters
    C.skipSpace
    pure (before, instr, after)
    
samples = parsed (C.many' parseSample) part1

In [5]:
import Data.Bits

addr, addi :: V.Vector Int -> V.Vector Int -> V.Vector Int

oprr op rs is = let
    opA = rs V.! (is V.! 1)
    opB = rs V.! (is V.! 2)
    rsC = op opA opB
    in rs V.// [(is V.! 3, rsC)]

opri op rs is = let
    opA = rs V.! (is V.! 1)
    opB = (is V.! 2)
    rsC = op opA opB
    in rs V.// [(is V.! 3, rsC)]

opir op rs is = let
    opA = (is V.! 1)
    opB = rs V.! (is V.! 2)
    rsC = op opA opB
    in rs V.// [(is V.! 3, rsC)]

fromBool False = 0
fromBool True = 1

(addr, addi) = (oprr (+), opri (+))
(mulr, muli) = (oprr (*), opri (*))
(banr, bani) = (oprr (fromIntegral .: (.&.)), opri (.&.))
(borr, bori) = (oprr (fromIntegral .: (.|.)), opri (.|.))
(setr, seti) = (oprr const, opir const)
(gtrr, gtri, gtir) = (oprr (fromBool .: (>)), opri (fromBool .: (>)), opir (fromBool .: (>)))
(eqrr, eqri, eqir) = (oprr (fromBool .: (==)), opri (fromBool .: (==)), opir (fromBool .: (==)))

In [6]:
matches (rs,is,as) instr = (instr rs is) == as

instrs = [addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtrr, gtri, gtir, eqrr, eqri, eqir]

matchCount = [length (filter id (map (matches m) instrs)) | m <- samples]

length $ filter (>=3) matchCount

612

In [7]:
import Control.Applicative

matchInstrs = [map (matches m) instrs | m <- samples]

zipped = zip (map (\(b,i,a) -> V.head i) samples) matchInstrs

fours = filter (\z -> fst z == 4) zipped

In [8]:
table = map (\i -> foldr1 (\(n, ms) (_, ms') -> (n, zipWith (&&) ms ms')) $ filter (\z -> fst z == i) zipped) [0..15]

In [9]:
import Data.List

table' = map (\(i, ms) -> (i, V.fromList (map fromBool ms))) table

solve :: [(Int, V.Vector Int)] -> [(Int, Maybe Int)] -> [(Int, Maybe Int)]
solve []         found = found
solve candidates found = let
    (narrowed, multiple) = partition (\(_, ms) -> sum ms == 1) candidates
    indices = map (\(i, ms) -> (i, V.elemIndex 1 ms)) narrowed
    toRemove = map (\(_, Just i) -> (i, 0)) indices
    multiple' = map (\(i, v) -> (i,v V.// toRemove)) multiple
    found' = indices ++ found
    in solve multiple' found'

solved = solve table' []

In [10]:
import qualified Data.Map.Strict as Map

instrMap = Map.fromList $ map (\(i, Just ix) -> (i, instrs !! ix )) solved

In [11]:
program = parsed (C.many' (C.skipSpace *> parseInstr)) part2

step rs is = let
    instr = instrMap Map.! V.head is
    in instr rs is

In [12]:
foldl' step (V.fromList [0,0,0,0]) program

[485,3,6,485]